In [78]:
# IMPORT PACKAGES
from astropy.io import fits
from ctypes import *
import matplotlib.pyplot as plt
import numpy as np # NumPy
#import os
from pysiril.siril import *
from pysiril.wrapper import *
#import sys
import time

In [2]:
app = Siril("/usr/bin/siril")
cmd = Wrapper(app)
app.Open()

INFO   : VERSION siril 1.2.0 : 
INFO   : Siril is compatible with pysiril
INFO   : Initialisation pySiril V0.0.15 : OK
INFO   : First step: Starting 'Siril -p' ....
INFO   : ThreadSiril is started
INFO   : run : /usr/bin/siril -p
INFO   : waiting: 
7s 
INFO   : Second step: Starting pipe reader ....
INFO   : ThreadReader is started
INFO   : Third step: Starting pipe writer ....
INFO   : PipeWriter is started
INFO   : Open()
> requires 1.2.0
: ready
: log: Running command: requires
> [status: success]


True

In [4]:
# CONVERT FILETYPES
cmd.cd("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Bias/")
cmd.convertraw(basename="Bias", debayer=True, fitseq=True, out="/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Bias/")

cmd.cd("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Dark/")
cmd.convertraw(basename="Dark", debayer=True, fitseq=True, out="/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Dark/")

cmd.cd("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Flat/")
cmd.convertraw(basename="Flat", debayer=True, fitseq=True, out="/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Flat/")

cmd.cd("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Light/")
cmd.convertraw(basename="Light", debayer=True, fitseq=True, out="/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Light/")

: log: Filter pattern: RGGB
: log: Filter Pattern: RGGB
: log: Reading RAW: file DSC_0108.NEF, 3 layer(s), 4948x3280 pixels
: log: Saving FITS: file /home/aanishpradhan/GitHub/cuSiril/NGC-1499/Flat/Dark_00106.fit, 3 layer(s), 4948x3280 pixels, 16 bits
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.3 sec)
: log: Filter pattern: RGGB
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.3 sec)
: log: Filter pattern: RGGB
: log: Filter Pattern: RGGB
: log: Filter Pattern: RGGB
: log: Saving FITS: file /home/aanishpradhan/GitHub/cuSiril/NGC-1499/Flat/Dark_00108.fit, 3 layer(s), 4948x3280 pixels, 16 bits
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.3 sec)
: log: Filter pattern: RGGB
: log: Filter Pattern: RGGB
: log: Reading RAW: file DSC_0110.NEF, 3 layer(s), 4948x3280 pixels
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.3 sec)
: log: Filter pattern: RGGB
: log: Filter Pattern: RGGB
: log: Reading RAW: file DSC_0109.NEF, 3 layer(s), 4948x3280 pixels
: log: R

(True,)

In [6]:
help(cmd.stack)

Help on method stack in module pysiril.wrapper:

stack(sequencename, type=None, rejection_type=None, sigma_low=3, sigma_high=3, norm=None, output_norm=False, rgb_equal=False, out=None, filter_fwhm=None, filter_wfwhm=None, filter_round=None, filter_bkg=None, filter_nbstars=None, filter_quality=None, filter_included=False, filter_incl=False, weight_from_noise=False, weight_from_nbstack=False, weight_from_nbstars=False, weight_from_wfwhm=False, fastnorm=False, rejmap=False, rejmaps=False) method of pysiril.wrapper.Wrapper instance
    Stacks the seqfilename sequence, using options.
    
    Syntax     : status=cmd.stack(seqfilename)
                 status=cmd.stack(seqfilename, type={sum|min|max}, [filtering] [output_norm] [weighted] [out=filename])
                 status=cmd.stack(seqfilename, type={med|median},  [norm=no, norm=] [filter-included] [weighted] [out=filename])
                 status=cmd.stack(seqfilename, type={rej|mean}, [rejection_type], sigma_low, sigma_high, [nonorm,

In [8]:
# STACK BIAS FRAME
def masterBias(biasDirectory):
	cmd.cd(biasDirectory)
	cmd.stack("Bias", type = "sum", norm = "no")
masterBias("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Bias/")

: log: Running command: stack
: log: Checking sequences in the directory: /home/aanishpradhan/GitHub/cuSiril/NGC-1499/Bias/.
: log: Stacking sequence Bias
: log: Not using rejection for stacking
: log: Processing all images in the sequence (5)
: log: Stacking result will be stored as a 32-bit image
: log: Sum stacking: with the current memory and thread limits, up to 5 thread(s) can be used
: log: Sum stacking: processing...
: log: Sequence processing succeeded.
: log: Execution time: 728.93 ms
: log: Integration of 5 images on 5 of the sequence:
: log: Pixel combination ......... normalized sum
: log: Input normalization ....... none
: log: Output normalization ...... disabled
: log: Pixel rejection ........... none
: log: Image weighting ........... disabled
: log: RGB equalization .......... disabled
: log: Background noise value (channel: #0): 936.074 (1.428e-02)
: log: Background noise value (channel: #1): 766.317 (1.169e-02)
: log: Background noise value (channel: #2): 946.968 (1

In [46]:
# View bias frames data
biasFrame = fits.open("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Bias/Bias.fit")
print("Total bias frames:", len(biasFrame))
print("Image size:", np.shape(biasFrame[0].data.T))
print("Data type:", biasFrame[0].data.dtype)

biasFrameRedChannel = np.empty((4948, 3280, 5), dtype = "uint16")
biasFrameGreenChannel = np.empty((4948, 3280, 5), dtype = "uint16")
biasFrameBlueChannel = np.empty((4948, 3280, 5), dtype = "uint16")

for subframe in range(0, len(biasFrame)):
	biasFrameRedChannel[:, :, subframe] = (biasFrame[subframe].data.T)[:, :, 0]
	biasFrameGreenChannel[:, :, subframe] = (biasFrame[subframe].data.T)[:, :, 1]
	biasFrameBlueChannel[:, :, subframe] = (biasFrame[subframe].data.T)[:, :, 2]

Total bias frames: 5
Image size: (4948, 3280, 3)
Data type: uint16


In [66]:
# View master bias frame data
masterBiasFrame = fits.open("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Bias/Bias_stacked.fit")
print("Total frames:", len(masterBiasFrame)) # Should be 1
print("Image size:", np.shape(masterBiasFrame[0].data.T)) # Should be same as above
print("Data type:", masterBiasFrame[0].data.dtype) # Big endian float32

Total frames: 1
Image size: (4948, 3280, 3)
Data type: >f4


In [161]:
red = biasFrameRedChannel[:, :, 0] + biasFrameRedChannel[:, :, 1] + biasFrameRedChannel[:, :, 2] + \
	biasFrameRedChannel[:, :, 3] + biasFrameRedChannel[:, :, 4]
a = np.max(red)
print(red)
print(a)

[[ 26  26  45 ... 100  61  22]
 [ 18  18  48 ...  83  62  39]
 [ 10  10  48 ...  66  61  56]
 ...
 [ 68  68  75 ...  35  30  23]
 [ 94  94  83 ...  26  28  30]
 [ 94  94  83 ...  26  28  30]]
1374


In [162]:
green = biasFrameGreenChannel[:, :, 0] + biasFrameGreenChannel[:, :, 1] + biasFrameGreenChannel[:, :, 2] + \
	biasFrameGreenChannel[:, :, 3] + biasFrameGreenChannel[:, :, 4]
b = np.max(green)
print(green)
print(b)

[[38 21 12 ... 27 20 44]
 [40 14 52 ...  0 48 68]
 [68 54 86 ... 39 98 55]
 ...
 [38 64 60 ... 22 33 42]
 [18 53 40 ... 45 36 41]
 [52 86 51 ... 70 52 46]]
1206


In [163]:
blue = biasFrameBlueChannel[:, :, 0] + biasFrameBlueChannel[:, :, 1] + biasFrameBlueChannel[:, :, 2] + \
	biasFrameBlueChannel[:, :, 3] + biasFrameBlueChannel[:, :, 4]
c = np.max(blue)
print(blue)
print(c)

[[52 40 28 ... 60 20 20]
 [52 40 28 ... 60 20 20]
 [42 54 62 ... 46 18 18]
 ...
 [ 4 24 44 ... 15 30 30]
 [41 35 27 ... 23 15 15]
 [78 44 10 ... 27  0  0]]
1206


In [165]:
np.linalg.norm(masterBiasFrame[0].data.T[:, :, 0] - (red / a))

3.81276802429149e-06

In [167]:
np.linalg.norm(masterBiasFrame[0].data.T[:, :, 1] - (green / a))

3.0267585463426106e-06

In [168]:
np.linalg.norm(masterBiasFrame[0].data.T[:, :, 2] - (blue / a))

3.65359647572094e-06

In [9]:
# STACK DARK FRAME
def masterDark(darkDirectory):
	cmd.cd(darkDirectory)
	cmd.stack("Dark", type = "sum", norm = "no")
masterDark("/home/aanishpradhan/GitHub/cuSiril/NGC-1499/Dark/")

: log: Running command: stack
: log: Stacking sequence Bias
: log: Not using rejection for stacking
: log: Processing all images in the sequence (5)
: log: Stacking result will be stored as a 32-bit image
: log: Sum stacking: with the current memory and thread limits, up to 5 thread(s) can be used
: log: Sum stacking: processing...
: log: Sequence processing succeeded.
: log: Execution time: 695.69 ms
: log: Integration of 5 images on 5 of the sequence:
: log: Pixel combination ......... normalized sum
: log: Input normalization ....... none
: log: Output normalization ...... disabled
: log: Pixel rejection ........... none
: log: Image weighting ........... disabled
: log: RGB equalization .......... disabled
: log: Background noise value (channel: #0): 936.074 (1.428e-02)
: log: Background noise value (channel: #1): 766.317 (1.169e-02)
: log: Background noise value (channel: #2): 946.968 (1.445e-02)
: log: Saving FITS: file Bias_stacked.fit, 3 layer(s), 4948x3280 pixels, 32 bits
: lo